In [39]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [40]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [41]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [42]:
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [43]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_counts[app_counts <= 10].index)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [44]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [45]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x >1]


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [46]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts <= 10].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
Other      118
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [47]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies_df = pd.get_dummies(application_df)

In [48]:
# Split our preprocessed data into our features and target arrays
y = app_dummies_df['IS_SUCCESSFUL']
X = app_dummies_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [49]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [50]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input = len(X_train_scaled[0])

hidden_nodes_layer1 = 5
hidden_nodes_layer2 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=num_input))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 5)                 350       
                                                                 
 dense_17 (Dense)            (None, 50)                300       
                                                                 
 dense_18 (Dense)            (None, 1)                 51        
                                                                 
Total params: 701 (2.74 KB)
Trainable params: 701 (2.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [51]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [52]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6050 - accuracy: 0.7013
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5686 - accuracy: 0.7248
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5620 - accuracy: 0.7271
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5582 - accuracy: 0.7281
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5550 - accuracy: 0.7287
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7294
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5503 - accuracy: 0.7290
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7294
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7313
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5484 - accura

In [53]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5525 - accuracy: 0.7259 - 452ms/epoch - 2ms/step
Loss: 0.552486002445221, Accuracy: 0.7259474992752075


In [54]:
# Export our model to HDF5 file
nn.save('../content/sample_data/AlphabetSoupCharity_Optimization_1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [55]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input = len(X_train_scaled[0])

hidden_nodes_layer1 = 5
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 75

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=num_input))

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 5)                 350       
                                                                 
 dense_20 (Dense)            (None, 50)                300       
                                                                 
 dense_21 (Dense)            (None, 75)                3825      
                                                                 
 dense_22 (Dense)            (None, 1)                 76        
                                                                 
Total params: 4551 (17.78 KB)
Trainable params: 4551 (17.78 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [56]:
# Compile the model
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [57]:
# Train the model
fit_model = nn_2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5968 - accuracy: 0.6970
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5650 - accuracy: 0.7186
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5607 - accuracy: 0.7227
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5583 - accuracy: 0.7231
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5558 - accuracy: 0.7234
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5556 - accuracy: 0.7254
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5542 - accuracy: 0.7260
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5536 - accuracy: 0.7268
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5517 - accuracy: 0.7290
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5505 - accura

In [58]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5536 - accuracy: 0.7236 - 471ms/epoch - 2ms/step
Loss: 0.5536107420921326, Accuracy: 0.7236151695251465


In [59]:
# Export our model to HDF5 file
nn_2.save('../content/sample_data/AlphabetSoupCharity_Optimization_2.h5')

In [60]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input = len(X_train_scaled[0])

hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 30

nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=num_input))

# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_3.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 10)                700       
                                                                 
 dense_24 (Dense)            (None, 30)                330       
                                                                 
 dense_25 (Dense)            (None, 1)                 31        
                                                                 
Total params: 1061 (4.14 KB)
Trainable params: 1061 (4.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
# Compile the model
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [62]:
# Train the model
fit_model = nn_3.fit(X_train_scaled, y_train, epochs=80)

Epoch 1/80
804/804 [==============================] - 3s 2ms/step - loss: 0.5946 - accuracy: 0.7070
Epoch 2/80
804/804 [==============================] - 1s 2ms/step - loss: 0.5592 - accuracy: 0.7259
Epoch 3/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5543 - accuracy: 0.7276
Epoch 4/80
804/804 [==============================] - 1s 2ms/step - loss: 0.5517 - accuracy: 0.7290
Epoch 5/80
804/804 [==============================] - 1s 2ms/step - loss: 0.5496 - accuracy: 0.7307
Epoch 6/80
804/804 [==============================] - 1s 2ms/step - loss: 0.5485 - accuracy: 0.7319
Epoch 7/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7320
Epoch 8/80
804/804 [==============================] - 2s 3ms/step - loss: 0.5469 - accuracy: 0.7316
Epoch 9/80
804/804 [==============================] - 1s 2ms/step - loss: 0.5461 - accuracy: 0.7327
Epoch 10/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5452 - accuracy: 0.7337

In [63]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5520 - accuracy: 0.7291 - 429ms/epoch - 2ms/step
Loss: 0.5520228743553162, Accuracy: 0.7290962338447571


In [64]:
# Export our model to HDF5 file
nn_3.save('../content/sample_data/AlphabetSoupCharity_Optimization_3.h5')